In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, MaxPooling1D, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, CSVLogger
import keras_metrics

from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
import numpy as np
import pickle

In [5]:
from cleaner import Cleaner 
cleaner = Cleaner()
cleaner.create_tokenizer_and_clean()

cleaning the text data
loading toikenizer


In [17]:
filename = "../data/json_bundle_reviews/large-bundle-clean.json"
df = pd.read_json(filename)
#df = self.clean_news(df)

y = df.sentiment.values
sentences = df['content'].values
sentences.shape


(100836,)

In [18]:
sentences_train, sentences_test, y_train, y_test = train_test_split(
sentences, y, test_size=0.25, random_state=1000)
maxlen = 250
with open('../data/neural_network_config/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [19]:
filter_length = 300

        #model = Sequential()
#model.add(Embedding(vocab_size, 20, input_length=maxlen))
#model.add(Dropout(0.15))
#model.add(GlobalMaxPool1D())
#model.add(Dense(output_size, activation='sigmoid'))
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=maxlen))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(Conv1D(filter_length, 5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy',keras_metrics.precision(), keras_metrics.recall()])

In [ ]:
print("training model")
csv_logger = CSVLogger('log_loss.csv', append=False, separator=';')

callbacks = [
ModelCheckpoint(filepath='../data/neural_network_config/temp-model.h5', save_best_only=True), csv_logger]

history = model.fit(X_train, y_train,
                    epochs=40,
                    batch_size=100,
                    validation_data=(X_test, y_test),
                    callbacks=callbacks)


In [ ]:
print("saving model")
# serialize model to JSON
model_json = model.to_json()
with open("../data/neural_network_config/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../data/neural_network_config/model.h5")
print("Saved model to disk")